# graph

> TODO fill in description

In [ ]:
#| default_exp graph

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from dataclasses import dataclass, replace
from typing import Type, Optional, Union, Any, List
from collections.abc import Hashable
from types import MappingProxyType
from enum import Enum
import copy
from IPython.display import Markdown

import fbdev
from fbdev.utils import AttrContainer
from fbdev.port import PortType, PortSpec, PortTypeSpec, PortSpecCollection
from fbdev.component import BaseComponent

In [ ]:
#|hide
from fbdev.component import BaseComponent

In [ ]:
#|hide
show_doc(fbdev.graph.EdgeSpec)

---

### EdgeSpec

>      EdgeSpec (maxsize:int=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
@dataclass(frozen=True)
class EdgeSpec:
    pass

In [ ]:
#|export
class EdgeSpec:
    def __init__(self,
                 maxsize:int=None):
        self._id = None
        self.maxsize = maxsize
        
    @property
    def id(self): return self._id
    
    @property
    def finite_maxsize(self):
        return self.maxsize is not None
    
    def copy(self):
        copy = copy.copy(self)
        return copy

In [ ]:
#|hide
show_doc(fbdev.graph.NodeSpec)

---

### NodeSpec

>      NodeSpec (component_type:Type[fbdev.component.BaseComponent])

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class NodeSpec:
    def __init__(self, component_type:Type[BaseComponent]):
        self._id = None
        self._component_type = component_type
        
    @property
    def id(self): return self._id
    @property
    def name(self):
        if self.id is None:
            _id = Graph.GRAPH_ID
        elif self.id==self.component_name:
            _id = ""
        elif self.id.startswith(self.component_name):
            _id = "..." + self.id[len(self.component_name):]
        else:
            _id = self.id
        return f"Node({self.component_name})[{_id}]"
    @property
    def component_type(self): return self._component_type
    @property
    def component_name(self) : return self._component_type.__name__
    @property
    def port_specs(self): return self._component_type.port_specs
    
    @property
    def contains_graph(self): return self._component_type.contains_graph
    
    def copy(self):
        copy = copy.copy(self)
        copy._component_type = self._component_type
        return copy

In [ ]:
#|hide
show_doc(fbdev.graph.Graph)

---

### Graph

>      Graph (port_spec_collection:fbdev.port.PortSpecCollection)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
a = {'aa': 3, 'b': 2, 'a': 5}

sum(1 for key in a if key.startswith('a'))

2

In [ ]:
#|export
class Graph:
    GRAPH_ID='net'
    
    def __init__(self, port_spec_collection:PortSpecCollection):
        self._port_specs = port_spec_collection
        self._edges = {}
        self._nodes = {}
        self._head_connections = {}
        self._tail_connections = {}
        self._edge_to_head_connections = {}
        self._edge_to_tail_connections = {}
        
    @property
    def component_type(self): raise RuntimeError("Graphs do not have a component type.")
    @property
    def port_specs(self): return self._port_specs
    @property
    def internal_port_connections(self): return self._internal_port_connections
    @property
    def nodes(self): return MappingProxyType(self._nodes)
    @property
    def edges(self): return MappingProxyType(self._edges)
    @property
    def head_connections(self): return MappingProxyType(self._head_connections)
    @property
    def tail_connections(self): return MappingProxyType(self._tail_connections)
    @property
    def graph_head_connections(self):
        return MappingProxyType({k[1:]:v for k,v in self.head_connections.items() if k[0] == self.GRAPH_ID})
    @property
    def graph_tail_connections(self):
        return MappingProxyType({k[1:]:v for k,v in self.tail_connections.items() if k[0] == self.GRAPH_ID})
    @property
    def edge_to_head_connections(self): return MappingProxyType(self._edge_to_head_connections)
    @property
    def edge_to_tail_connections(self): return MappingProxyType(self._edge_to_tail_connections)
        
    def add_node(self, node, id=None):
        if id == self.GRAPH_ID: raise RuntimeError(f"Node id '{self.GRAPH_ID}' is reserved for the graph itself.")
        if id:
            node._id = id
        else:
            num_twins = sum(1 for _node in self.nodes.values() if _node.component_type == node.component_type)
            if num_twins > 0:
                node._id = f"{node.component_type.__name__}_{num_twins}"
            else:
                node._id = node.component_type.__name__
        if node.id in self.nodes: raise RuntimeError(f"Node '{node.id}' already exists.")
        self._nodes[node.id] = node
        return node.id
        
    def add_edge(self, edge: EdgeSpec, id=None):
        edge._id = id or len(self.edges)
        if edge.id in self.edges: raise RuntimeError(f"Edge '{edge.id}' already exists.")
        self._edges[edge.id] = edge
        return edge.id
        
    def remove_node(self, node_id):
        if node_id not in self.nodes: raise RuntimeError(f"Node {node_id} is not in this graph.")
        node = self.nodes[node_id]
        for port_type, port_name in node.port_specs.get_port_names():
            if (node_id, port_type, port_name) in self._head_connections or (node_id, port_type, port_name) in self._tail_connections:
                self.disconnect_edge_from_node(node_id, port_type, port_name)
        del self.nodes[node_id]
        
    def remove_edge(self, edge_id):
        if edge_id not in self.edges: raise RuntimeError(f"Edge {edge_id} is not in this graph.")
        if edge_id in self._edge_to_head_connections:
            self.disconnect_edge_from_node(*self._edge_to_head_connections[edge_id])
        if edge_id in self._edge_to_tail_connections:
            self.disconnect_edge_from_node(*self._edge_to_tail_connections[edge_id])
        del self.edges[edge_id]
        
    def connect_edge_to_node(self, node_id:Hashable, port_type: Union[PortType, str], port_name: str, edge_id:Hashable):
        node_is_graph = node_id == self.GRAPH_ID
        node = self if node_is_graph else self.nodes[node_id]
        edge = self.edges[edge_id]
        if type(port_type) == str: port_type = PortType.get(port_type)
        is_head_connection = port_type.is_input_port if not node_is_graph else not port_type.is_input_port
        if (port_type, port_name) not in node.port_specs:
            if not node_is_graph:
                raise RuntimeError(f"Port {port_type.label}.{port_name} does not exist in {node.component_type.__name__}.")
            else:
                raise RuntimeError(f"Port {port_type.label}.{port_name} does not exist in graph port specs.")
        connections = self._head_connections if is_head_connection else self._tail_connections
        edge_to_connections = self._edge_to_head_connections if is_head_connection else self._edge_to_tail_connections
        if (node_id,port_type,port_name) in connections:
            raise RuntimeError(f"Edge {connections[(node_id,port_type,port_name)]} already connected to port {port_type.label}.{port_name} of Node {node_id}.")
        if edge_id in edge_to_connections:
            raise RuntimeError(f"Edge {edge_to_connections[edge_id]} already connected to port {port_type.label}.{port_name} of Node {node_id}.")
        connections[(node_id,port_type,port_name)] = edge_id
        edge_to_connections[edge_id] = (node_id,port_type,port_name)

    def disconnect_edge_from_node(self, node_id:Hashable, port_type: Union[PortType, str], port_name: str):
        node_is_graph = node_id == self.GRAPH_ID
        node = self if node_is_graph else self.nodes[node_id]
        if type(port_type) == str: port_type = PortType.get(port_type)
        is_head_connection = port_type.is_input_port if not node_is_graph else not port_type.is_input_port
        if not (port_type.label, port_name) in node.port_specs:
            if not node_is_graph:
                raise RuntimeError(f"Port {port_type.label}.{port_name} does not exist in {node.component_type.__name__}.")
            else:
                raise RuntimeError(f"Port {port_type.label}.{port_name} does not exist in graph port specs.")
        connections = self._head_connections if is_head_connection else self._tail_connections
        edge_to_connections = self._edge_to_head_connections if is_head_connection else self._edge_to_tail_connections
        if (node_id,port_type,port_name) not in connections:
            raise RuntimeError(f"Port {port_type.label}.{port_name} of Node {node_id} is not connected to an edge.")
        edge_id = connections[(node_id,port_type,port_name)]
        del connections[(node_id,port_type,port_name)]
        del edge_to_connections[edge_id]
        
    def connect_edge_to_graph_port(self, port_type: PortType, port_name: str, edge_id:Hashable):
        self.connect_edge_to_node(self.GRAPH_ID, port_type, port_name, edge_id)
        
    def disconnect_edge_from_graph_port(self, port_type: PortType, port_name: str):
        self.disconnect_edge_from_graph_port(self.GRAPH_ID, port_type, port_name)
            
    def connect_nodes(self, node1_id:Hashable, port_type1: Union[PortType, str], port_name1: str,
                      node2_id:Hashable, port_type2: Union[PortType, str], port_name2: str,
                      edge:EdgeSpec=None):
        edge = edge or EdgeSpec()
        edge_id = self.add_edge(edge)
        self.connect_edge_to_node(node1_id, port_type1, port_name1, edge_id)
        self.connect_edge_to_node(node2_id, port_type2, port_name2, edge_id)
        
    def connect_node_to_graph_port(self, node_id:Hashable, port_type: PortType, port_name: str,
                                   graph_port_type: PortType, graph_port_name: str,
                                   edge:EdgeSpec=None):
        edge = edge or EdgeSpec()
        self.connect_nodes(node_id, port_type, port_name, self.GRAPH_ID, graph_port_type, graph_port_name, edge)
            
    def copy(self):
        g = super().copy()
        g._port_specs = self._port_specs.copy()
        for node in self.nodes.values(): g.add_node(node.copy(), id=node.id)
        for edge in self.edges.values(): g.add_edge(edge.copy(), id=edge.id)
        g._head_connections = self._head_connections.copy()
        g._tail_connections = self._tail_connections.copy()
        return g

    def to_mermaid(self, flowchart_orientation:str=''):
        """
        TB - Top to bottom
        TD - Top-down/ same as top to bottom
        BT - Bottom to top
        RL - Right to left
        LR - Left to right
        """

        def tabs(add_tabs): return (add_tabs) * '    '
        def fmt(s): return s.replace('.', '__CHILD__').replace(':', '__PORTS__').replace('+', '__PROP__')
        port_type_to_shape = {
            PortType.INPUT: '{{%s}}',
            PortType.OUTPUT: '([%s])',
            PortType.CONFIG: '[(%s)]',
        }
        mermaid_classes = [
            'classDef subgraph_zone fill:#111',
            'classDef loose_edge fill:#f00',
        ]
        mermaid_subgraphs = []
        mermaid_edges = []
        mermaid_class_assignments = { l.split()[1] : [] for l in mermaid_classes }

        def _mermaid_helper(graph, graph_id, tab_level):
            def tabs(add_tabs): return (tab_level+add_tabs) * '    '
            
            for node_id, node in graph.nodes.items():
                if node_id == graph.GRAPH_ID:
                    raise ValueError("Found child node with `node_id=GRAPH_ID`")
                node_address = f"{graph_id}.{node_id}"
                mermaid_subgraphs.append(f'{tabs(0)}subgraph {fmt(node_address)}["{node.name}"]')
                
                # Subgraphs
                if node.contains_graph:
                    mermaid_subgraphs.append(f'{tabs(1)}subgraph {fmt(node_address+"+children")}[" "]')
                    _mermaid_helper(graph.nodes[node_id].component_type._graph, node_address, tab_level+2)
                    mermaid_subgraphs.append(f'{tabs(1)}end')
                    mermaid_class_assignments['subgraph_zone'].append(f"{node_address}+children")
                
                # Ports
                #if node.contains_graph: mermaid_subgraphs.append(f'{tabs(1)}subgraph {fmt(node_address+"+ports")}[" "]')
                for port_spec in node.port_specs.iter_ports():
                    address = f"{graph_id}.{node_id}:{port_spec.port_type.label}.{port_spec.name}"
                    name = port_spec.name
                    title = port_type_to_shape[port_spec.port_type] % name
                    tabn = 2 if node.contains_graph else 1
                    mermaid_subgraphs.append(f'{tabs(tabn)}{fmt(address)}{title}')
                #if node.contains_graph: mermaid_subgraphs.append(f'{tabs(1)}end')
                    
                mermaid_subgraphs.append(f'{tabs(0)}end')
                        
            for edge_id, edge in graph.edges.items():
                head_conn = graph._edge_to_head_connections.get(edge_id)
                tail_conn = graph._edge_to_tail_connections.get(edge_id)
                
                if head_conn:
                    head_node, head_port_type, head_port_name = head_conn
                    if head_node == graph.GRAPH_ID: to_address = fmt(f"{graph_id}:{head_port_type.label}.{head_port_name}")
                    else: to_address = fmt(f"{graph_id}.{head_node}:{head_port_type.label}.{head_port_name}")
                if tail_conn:
                    tail_node, tail_port_type, tail_port_name = tail_conn
                    if tail_node == graph.GRAPH_ID: from_address = fmt(f"{graph_id}:{tail_port_type.label}.{tail_port_name}")
                    else: from_address = fmt(f"{graph_id}.{tail_node}:{tail_port_type.label}.{tail_port_name}")
                    
                if head_conn and tail_conn:
                    arrow = '-.->' if graph.GRAPH_ID in [tail_node, head_node] else '-->'
                    if edge.finite_maxsize: mermaid_edges.append(f"{from_address} {arrow}|{edge.maxsize}| {to_address}")
                    else: mermaid_edges.append(f"{from_address} {arrow} {to_address}")
                else:
                    if head_conn: mermaid_class_assignments['loose_edge'].append(to_address)
                    elif tail_conn: mermaid_class_assignments['loose_edge'].append(from_address)
    
        _mermaid_helper(self, self.GRAPH_ID, 1)
        
        mermaid_graph_ports = []
        for port_spec in self.port_specs.iter_ports():
            address = f"{self.GRAPH_ID}:{port_spec.port_type.label}.{port_spec.name}"
            name = port_spec.name
            title = port_type_to_shape[port_spec.port_type] % name
            mermaid_graph_ports.append(fmt(f'{tabs(2)}{address}{title}'))
            
        mermaid_classes = list(map(lambda l: f"{tabs(1)}{l}", mermaid_classes))
        mermaid_edges = list(map(lambda l: f"{tabs(1)}{l}", mermaid_edges))
        mermaid_class_assignments = {class_name : list(map(fmt, mermaid_class_assignments[class_name])) for class_name in mermaid_class_assignments}
        _mermaid_class_assignments = [f"{tabs(1)}class {','.join(mermaid_class_assignments[class_name])} {class_name}" for class_name in mermaid_class_assignments if mermaid_class_assignments[class_name]]
        
        mermaid = [f"flowchart {flowchart_orientation}"] + mermaid_classes + [""] + mermaid_subgraphs + [""] + mermaid_graph_ports + [""] + mermaid_edges + [""] + _mermaid_class_assignments
        return "\n".join(mermaid)
    
    def display_mermaid(self):
        return Markdown(f"```mermaid\n{self.to_mermaid()}\n```")
    
    def _repr_markdown_(self): return f"```mermaid\n{self.to_mermaid()}\n```"
    
    def is_DAG(self):
        visited = set()
        rec_stack = set()
        
        def is_cyclic(node_id:int):
            # Mark the current node as visited and add it to the recursion stack
            visited.add(node_id)
            rec_stack.add(node_id)
            
            if node_id == self.GRAPH_ID:
                port_specs = self.port_specs
            else:
                port_specs = self.nodes[node_id].port_specs
            
            for port_type, port_name in port_specs.get_port_names():
                if (node_id, port_type, port_name) in self._tail_connections:
                    edge_id = self.tail_connections[(node_id, port_type, port_name)]
                    if edge_id in self._edge_to_head_connections:
                        target_node_id, _, _ = self._edge_to_head_connections[edge_id]
                        if target_node_id not in visited:
                            if is_cyclic(target_node_id):
                                return True
                        elif target_node_id in rec_stack and target_node_id != self.GRAPH_ID: # Graph does not count
                            return True
                    
            rec_stack.remove(node_id)
            return False
        
        node_ids = list(self.nodes.keys())
        node_ids += [v[0] for v in self.tail_connections.keys()]
        node_ids += [v[0] for v in self.head_connections.keys()]
        node_ids = set(node_ids)

        for node_id in node_ids:
            if node_id not in visited:
                if is_cyclic(node_id):
                    return False
        return True

In [ ]:
#|hide
show_doc(fbdev.graph.ReadonlyGraph)

---

### ReadonlyGraph

>      ReadonlyGraph (graph:fbdev.graph.Graph)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class ReadonlyGraph:
    def __init__(self, graph: Graph):
        self._graph = graph
        
    @property
    def nodes(self): return self._graph.nodes
    @property
    def edges(self): return self._graph.edges
    @property
    def head_connections(self): return self._graph.head_connections
    @property
    def tail_connections(self): return self._graph.tail_connections
    @property
    def edge_to_head_connections(self): return self._graph.edge_to_head_connections
    @property
    def edge_to_tail_connections(self): return self._graph.edge_to_tail_connections
    @property
    def graph_head_connections(self): return self._graph.graph_head_connections
    @property
    def graph_tail_connections(self): return self._graph.graph_tail_connections
    
    def to_mermaid(self): return self._graph.to_mermaid()
    def display_mermaid(self): return self._graph.display_mermaid()
    def _repr_markdown_(self): return self._graph._repr_markdown_()
    def is_DAG(self): return self._graph.is_DAG()
    def copy(self) -> Graph: return self._graph.copy()